1. Load the Required Libraries

In [1]:
import pandas as pd
import numpy as np
import dtale
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_auc_score,recall_score

C:\Users\Frank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\dtale\utils.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


2. Read the data from the dataset

In [ ]:
df = pd.read_csv("C:\\Users\\Frank\\OneDrive\\Documentos\\ResearchPapers\\Datasets\\diabetes_binary_health_indicators_BRFSS2015.csv", encoding='latin1')
dtale.show(df)

In [3]:
df.head()
df.drop(columns=['CholCheck','Smoker','Fruits','Veggies','HvyAlcoholConsump','AnyHealthcare','NoDocbcCost','MentHlth','Sex','Education'], inplace=True)

3. Split the attribites into dependent and independent attributes

In [4]:
X = df.iloc[:, 1:].values
Y = df.iloc[:, 0].values
dtale.show(Y, ignore_duplicate=True)

3. Handling missing values and replacing missing values with nan from numpy and replace with mean of all the other values

In [ ]:
imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
cols_to_impute= [1,2,3,4,5,6,7,8,9,10]  # Fit the imputer on the data excluding the first column
X[:,cols_to_impute] = imputer.fit_transform(X[:,cols_to_impute])
dtale.show(X, ignore_duplicate=True)

4. Encoding the categorical data

In [6]:
#cols_to_encode = [3]
# One-hot encode the categorical column
#ct = ColumnTransformer(
    #transformers=[
        #("onehot", OneHotEncoder(), cols_to_encode)  # Correct syntax
    #],
    #remainder="passthrough"
#)
#X = ct.fit_transform(X)  # Returns a sparse matrix (use .toarray() if needed)

# Encode target variable Y (ensure it's 1D)
#labelencoder_Y = LabelEncoder()
#Y = labelencoder_Y.fit_transform(Y.ravel())

5. Splitting the dataset intro training set and test set

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

6. Feature Scaling

In [8]:
dtale.show(X_test)

7. Train the Random Forest Model

In [9]:
rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
rf.fit(X_train, Y_train)

Kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_train, Y_train, cv=Kfold, scoring='accuracy')

Y_pred = rf.predict(X_test)

In [10]:
rf.score(X_test, Y_test)
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

         0.0       0.86      1.00      0.92     43525
         1.0       0.00      0.00      0.00      7211

    accuracy                           0.86     50736
   macro avg       0.43      0.50      0.46     50736
weighted avg       0.74      0.86      0.79     50736



C:\Users\Frank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Frank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Frank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa

In [11]:
rf2 = RandomForestClassifier(
    
    n_estimators=1000,
    criterion= 'entropy',
    min_samples_split= 10,
    random_state=42)

8. Retraining the model using Resampled data

In [12]:
smote_enn = SMOTE(sampling_strategy='minority', random_state=42)
X_train_res, Y_train_res = smote_enn.fit_resample(X_train, Y_train)
print(pd.Series(Y_train_res).value_counts())

0.0    174809
1.0    174809
Name: count, dtype: int64


In [13]:
model = RandomForestClassifier( random_state= 42)
model.fit(X_train_res, Y_train_res) 
Kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(model, X_train_res, Y_train_res, cv=Kfold, scoring='accuracy')

In [14]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
sensitivity = recall_score(Y_test, y_pred, pos_label=1)
roc_auc = roc_auc_score(Y_test, y_proba)

tn, fp, fn, tp = confusion_matrix(Y_test, y_pred).ravel()
specificity = tn / (tn + fp)

report = classification_report(Y_test, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()

pd.set_option("display.precision", 4)
print(df_report)
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Confusion Matrix:\n{confusion_matrix(Y_test, y_pred)}")
print(f"Mean Accuracy:{scores.mean():.4f} (+/- {scores.std():.4f})")

              precision  recall  f1-score     support
0.0              0.8843  0.9257    0.9045  43525.0000
1.0              0.3749  0.2688    0.3131   7211.0000
accuracy         0.8324  0.8324    0.8324      0.8324
macro avg        0.6296  0.5972    0.6088  50736.0000
weighted avg     0.8119  0.8324    0.8205  50736.0000
ROC-AUC: 0.7622
Sensitivity: 0.2688
Specificity: 0.9257
Confusion Matrix:
[[40293  3232]
 [ 5273  1938]]
Mean Accuracy:0.8983 (+/- 0.0015)
